In [ ]:
import ROOT
from ROOT import TFile
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
import copy
import pandas as pd 
import time
from os import listdir

In [ ]:
class particle:
    def __init__(self, pid, fourvector, virtual_photon,ThetaPQ):
        #work in frame that is aligned with photon
        #the photon is aligned with z-direction and has 4-momentum q= (0,0,sqrt(Nu2+Q2),Nu) (note q2 = -Q2)        
        self.virtual_photon = virtual_photon
        Nu = virtual_photon.E()
        Q2 = -virtual_photon.M2()
        self.proton = ROOT.TLorentzVector()
        self.proton.SetPxPyPzE(0,0,0, 0.938)
        self.W = (virtual_photon + self.proton).M()
        incoming_e = ROOT.TLorentzVector()
        incoming_e.SetPxPyPzE(0,0,5.014,5.014)
        part1 = virtual_photon.Vect().Cross(incoming_e.Vect()).Unit()
        part2 = virtual_photon.Vect().Cross(fourvector.Vect()).Unit()
        sign  = np.sign(part1.Dot(fourvector.Vect()))
        self.PhiPQ = sign*np.arccos(part1.Dot(part2))
        #print ' Angle my way ', self.PhiPQ
        #print 'Hayk (radians) ' , PhiPQ*np.pi/180.0
        
        photon_pz = np.sqrt(Nu*Nu+Q2) #direction is positive by definition
        self.bcm = photon_pz/(Nu + 0.938)#photon-nucleon center-of-mass velocity 
        self.ycm = 0.5*np.log(( 1+self.bcm)/(1-self.bcm)) #photon-nucleon center-of-mass rapidity
        
        self.LorentzVector = fourvector #hadron four-vector
        self.PhiLab = self.LorentzVector.Phi()
        self.E = self.LorentzVector.E() #energy in lab frame
        self.vector = self.LorentzVector.Vect()
        self.Pt = self.vector.Perp(virtual_photon.Vect().Unit()) #pT with respect to photon direction
        self.Pl  = self.vector.Dot(virtual_photon.Vect().Unit()) #pL with respect to photon direction (in lab frame)
        self.y =  0.5*np.log( (self.E+self.Pl)/(self.E-self.Pl)) #rapidity in lab frame
        self.mT = np.sqrt(self.LorentzVector.M2() + self.Pt*self.Pt)
        self.y_star = self.y - self.ycm
        self.Pl_star = self.mT*np.sinh(self.y_star)
        self.Xf = 2.0*self.Pl_star/self.W 
        self.pid = pid
        self.Zh = self.E/Nu
        self.ThetaPQ = np.arctan(self.Pt/self.Pl)
        #print 'My COSPQ' , np.arctan(self.Pt/self.Pl)
        #print 'Hayk COSPQ', self.ThetaPQ
        
    def redefine(self, new_virtual_photon):
        #this methods recalculates variables that depend on virtual photon direction
        #this helps for event mixing. 
        incoming_e = ROOT.TLorentzVector()
        incoming_e.SetPxPyPzE(0,0,5.014,5.014)
        part1 = new_virtual_photon.Vect().Cross(incoming_e.Vect()).Unit()
        part2 = new_virtual_photon.Vect().Cross(self.LorentzVector.Vect()).Unit()
       
        sign  = np.sign(part1.Dot(self.LorentzVector.Vect()))
        self.PhiPQ = sign*np.arccos(part1.Dot(part2)) 
        self.Pt = self.LorentzVector.Vect().Perp(new_virtual_photon.Vect().Unit()) #pT with respect to photon direction
        self.Pl  = self.LorentzVector.Vect().Dot(new_virtual_photon.Vect().Unit()) #pL with respect to photon direction (in lab frame)
        self.y =  0.5*np.log( (self.E+self.Pl)/(self.E-self.Pl)) #rapidity in lab frame
        self.ThetaPQ = np.arctan(self.Pt/self.Pl)
        self.virtual_photon = new_virtual_photon
        
        return
        
        
        
    def print_properties(self):
        print ('Hello, let me introduce myself, i am particle pid = ' , self.pid, ' with index ', self.index, ', from event  #', self.ievt, ' Nu and W', self.Nu, ' ' , self.W)
        print ('zh = ', self.Zh, ' phi_pq= ', self.PhiPQ, ' theta_pq=' , self.ThetaPQ, 'E = ', self.E, ' xf', self.Xf,'Pt ', self.Pt, ' Pl= ', self.Pl, ' rapidity=' ,  self.y)
        print ('pid = ' , self.pid)

In [ ]:

def getDataframes(filename, Target=1,maxevents=1e9,tree_name='tree_data'):
    dphi = np.array([])
    ParticlesFromPrevious = []
    try:
        myfile = TFile.Open('%s'%filename,'READ')
    except:
        print("could not open file")
        
    myfile.Print()
    mytree = myfile.Get(tree_name)
    e_tree = myfile.Get('e_rec')
        
    print (filename, ' has ', mytree.GetEntries(), ' entries')
    
    tupla_electron = {}
    tupla_electron = ['Q2']
    tupla_electron = ['Xb']
    tupla_electron = ['Nu']
    tupla_electron = ['W']

    
    tupla_hadron = {}
    tupla_hadron['pid'] = []
    tupla_hadron['xf'] = []
    tupla_hadron['xf_default'] = []
    tupla_hadron['z']  = []
    tupla_hadron['y']  = []
    tupla_hadron['ycm'] = []
    tupla_hadron['Q2'] = []
    tupla_hadron['Xb'] = []
    tupla_hadron['Nu'] = []
    tupla_hadron['W'] = []
    tupla_hadron['phi_pq'] = []
    tupla_hadron['theta_pq'] = []
    tupla_hadron['TargType'] = []
    tupla_hadron['mass'] = []
    tupla_hadron['phi_lab'] = []
    tupla_hadron['theta_lab'] = []
    tupla_hadron['pt'] = []
    
    start = time.time()

    for ievt  in range(mytree.GetEntries()):
        mytree.GetEntry(ievt)   
        e_tree.GetEntry(ievt)
        if mytree.W<2.05 or mytree.Q2<1.0: continue
        if ievt>maxevents: break
        if mytree.TargType!=Target: continue
        W = mytree.W
        Nu = mytree.Nu
        #get electron momentum:
        Pe = np.sqrt(e_tree.Pex*e_tree.Pex + e_tree.Pey*e_tree.Pey+ e_tree.Pez*e_tree.Pez)
        scattered_e = ROOT.TLorentzVector()
        scattered_e.SetPxPyPzE(e_tree.Pex, e_tree.Pey, e_tree.Pez, Pe)
        incoming_e = ROOT.TLorentzVector()
        incoming_e.SetPxPyPzE(0,0,5.014,5.014)
        virtual_photon  = incoming_e - scattered_e 
        virtual_photon_unitvector = virtual_photon.Vect().Unit()
        proton = ROOT.TLorentzVector()
        proton.SetPxPyPzE(0,0,0, 0.938)
        
        ##alternative calculation of virtual photon
        #scattered electron
        #E    = 5.014
        #Eprime = E-Nu
        #phi_e = -scattered_e.Vect().Phi()
        #alt_scattered_e = ROOT.TLorentzVector()
        #cos_thetae = 1-mytree.Q2/(2*E*Eprime)
        #sin_thetae = np.sqrt(1-cos_thetae*cos_thetae)
        #alt_scattered_e.SetPxPyPzE(Eprime*sin_thetae*np.cos(phi_e),
        #                       Eprime*sin_thetae*np.sin(phi_e),
        #                       Eprime*cos_thetae,
        #                       Eprime)
        #alt_virtual_photon  = incoming_e - alt_scattered_e
        
        #print 'Comparing normal and alternative virtual photons momentum'
        #print 'virtual ',virtual_photon.Px(), ' ', virtual_photon.Py(), ' ', virtual_photon.Pz(), ' ',virtual_photon.E()
        #print 'alt ' ,alt_virtual_photon.Px(), ' ', alt_virtual_photon.Py(), ' ',alt_virtual_photon.Pz(), ' ',alt_virtual_photon.E()
        #################################################
        
        
        
        #photon-nucleon center-of-mass energy and rapidity
        #the photon is aligned with z-direction and has 4-momentum q= (0,0,sqrt(Nu2+Q2),Nu) (note q2 = -Q2)
        
     
        #print 'W ',  W , 'electron W ', e_tree.W
        #print (virtual_photon + proton).M()
        #print ' Entering main loop over particles'
        for i in range(len(mytree.pid)):
            if abs(mytree.pid[i]) !=211: continue
            i_lv = ROOT.TLorentzVector()    
            i_lv.SetPxPyPzE(mytree.Px[i],mytree.Py[i],mytree.Pz[i],mytree.Zh[i]*Nu)
#            print i_lv.Phi(), '   ', mytree.Phi[i]*np.pi/180.0
            
            
            i_part = particle(mytree.pid[i], i_lv, virtual_photon, mytree.ThetaPQ[i] )
            #print i_part.LorentzVector.Phi()
            #print mytree.Phi[i]*np.pi/180.0
            #print i_part.LorentzVector.Theta(), '   ', mytree.Theta[i]*np.pi/180.0


            if i_part.Zh > 0.0: 
                tupla_hadron['TargType'].append(mytree.TargType)
                tupla_hadron['pid'].append(i_part.pid)
                tupla_hadron['xf'].append(i_part.Xf)
                tupla_hadron['xf_default'].append(mytree.Xf[i])
                tupla_hadron['z'].append(i_part.Zh)
                tupla_hadron['y'].append(i_part.y_star)
                tupla_hadron['ycm'].append(i_part.ycm)
                tupla_hadron['phi_pq'].append(i_part.PhiPQ)
                tupla_hadron['theta_pq'].append(i_part.ThetaPQ)
                tupla_hadron['mass'].append(i_part.LorentzVector.M())
                tupla_hadron['Q2'].append(mytree.Q2)
                tupla_hadron['Xb'].append(mytree.Xb)
                tupla_hadron['Nu'].append(mytree.Nu)
                tupla_hadron['W'].append(mytree.W)
                tupla_hadron['phi_lab'].append(i_part.LorentzVector.Phi())
                tupla_hadron['theta_lab'].append(mytree.Theta[i])
                tupla_hadron['pt'].append(i_part.Pt)
                
                #i_part.LorentzVector.Theta())
                #print 'mytree.Pt[i] ' , mytree.Pt[i], ' check: ' ,i_part.Vector.Perp(virtual_photon_unitvector)
                
        #print ' going for next event'    
        #print ' particles in event', len(particles
        ##end loop over events correlations    
    end = time.time()
    print ('Processed in',  end-start, 'seconds')
    
    df_electron = pd.DataFrame(tupla_electron)

    df_hadron = pd.DataFrame(tupla_hadron)
    myfile.Close()
    return df_electron,df_hadron

In [ ]:
#for a particle on-shell:
# E = mT cosh y 
# pz = mT sinh y
#mT2 = m2 + pt2
# rapidity in the ion-at-rest frame, with the photon direction is 
# y = 0.5 * ln( (E+pz)/(E-pz))
#rapidity of the target in the lab frame = 0 (at rest)
# y = y* + ycm
#bcm = az + bz/ a0 + b0 
#bcm = photon_z / Nu + m_{p}
#ycm = 0.5 * ln( 1+bcm/1-bcm)
#energy of the photon = Nu
#momentum of the photon (magnitude) = sqrt(Nu2 + Q2) 
#q2 = -Q2. (four-momentum transfer)

## Create dataframe df

In [ ]:
df = {}

## Pb data

In [ ]:
path = '/home/miguel/EG2_DATA/dihadronformat/Pb/'

Files = listdir(path) 
df['Pb_electron'], df['Pb_hadron'] = [None,None]
df['D_Pb_electron'], df['D_Pb_hadron'] = [None,None]

for name in Files:
    filename = path+name
    print (filename)
    e, h = getDataframes(filename,Target=2,maxevents=1e4)
    df['Pb_electron'] = pd.concat([ df['Pb_electron'], e])
    df['Pb_hadron'] = pd.concat([ df['Pb_hadron'], h])

    e,h = getDataframes(filename,Target=1,maxevents=1e4)
    df['D_Pb_electron'] = pd.concat([ df['D_Pb_electron'], e])
    df['D_Pb_hadron'] = pd.concat([ df['D_Pb_hadron'], h])


In [ ]:
df['Pb_hadron'].head()

## Fe Data

In [ ]:
path = '/mnt/c/Users/marratia/Linux/Fe_data/'

Files = listdir(path) 
df['Fe'],df['Fe_mix'], df['Fe_trigger'] = [None,None,None]
df['D_Fe'],df['D_Fe_mix'], df['D_Fe_trigger'] = [None,None,None]

for name in Files:
    filename = path+name
    pairs, pairs_mix, trigger = getDataframes(filename,Target=2)
    df['Fe'] = pd.concat([ df['Fe'], pairs])
    df['Fe_mix'] = pd.concat([ df['Fe_mix'], pairs_mix])
    df['Fe_trigger'] = pd.concat([ df['Fe_trigger'], trigger])
    pairs, pairs_mix, trigger = getDataframes(filename,Target=1)
    df['D_Fe'] = pd.concat([ df['D_Fe'], pairs])
    df['D_Fe_mix'] = pd.concat([ df['D_Fe_mix'], pairs_mix])
    df['D_Fe_trigger'] = pd.concat([ df['D_Fe_trigger'], trigger])

# C data

In [ ]:
path = '/mnt/c/Users/marratia/Linux/C_data/'
Files = listdir(path) 
df['C'],df['C_mix'], df['C_trigger'] = [None,None,None]
df['D_C'],df['D_C_mix'], df['D_C_trigger'] = [None,None,None]

for name in Files:
    filename = path+name
    pairs, pairs_mix, trigger = getDataframes(filename,Target=2)
    df['C'] = pd.concat([ df['C'], pairs])
    df['C_mix'] = pd.concat([ df['C_mix'], pairs_mix])
    df['C_trigger'] = pd.concat([ df['C_trigger'], trigger])
    pairs, pairs_mix, trigger = getDataframes(filename,Target=1)
    df['D_C'] = pd.concat([ df['D_C'], pairs])
    df['D_C_mix'] = pd.concat([ df['D_C_mix'], pairs_mix])
    df['D_C_trigger'] = pd.concat([ df['D_C_trigger'], trigger])

## MC

In [ ]:
path = '/mnt/c/Users/marratia/Linux/MC_C/'

Files = listdir(path) 
df['MC'],df['MC_mix'], df['MC_trigger'] = [None,None,None]
df['MC_true'],df['MC_true_mix'], df['MC_true_trigger'] = [None,None,None]

for name in Files:
    filename = path+name
    pairs, pairs_mix, trigger = getDataframes(filename,Target=2,tree_name='tree_accept')
    df['MC'] = pd.concat([ df['MC'], pairs])
    df['MC_mix'] = pd.concat([ df['MC_mix'], pairs_mix])
    df['MC_trigger'] = pd.concat([ df['MC_trigger'], trigger])

    pairs, pairs_mix, trigger = getDataframes(filename,Target=2,tree_name='tree_thrown')
    df['MC_true'] = pd.concat([ df['MC_true'], pairs])
    df['MC_true_mix'] = pd.concat([ df['MC_true_mix'], pairs_mix])
    df['MC_true_trigger'] = pd.concat([ df['MC_true_trigger'], trigger])



In [ ]:
import root_pandas
from root_pandas import read_root
from root_pandas import to_root 

## Save dataframes to ROOT files

In [ ]:
to_root(df['Pb'],'Single_Pb.root', key='Pb')
to_root(df['D_Pb'],'Single_Pb.root', key='D_Pb',mode='a')


In [ ]:
to_root(df['Fe'],'Single_Fe.root', key='Fe')
to_root(df['D_Fe'],'Single_Fe.root', key='D_Fe',mode='a')


In [ ]:
to_root(df['C'],'Single_C.root', key='C')
to_root(df['D_C'],'Single_C.root', key='D_C',mode='a')


In [ ]:
df['Pb_trigger'].hist(figsize=(12,12),bins=100)
plt.show()

In [ ]:
df['MC_true'].hist(figsize=(12,12),bins=100)
plt.show()

In [ ]:
df['Pb_mix'].hist(figsize=(12,12),bins=100)
plt.show()

## See distributions

In [ ]:
df['D_Pb_hadron'].hist()

In [ ]:
import matplotlib.colors as mcolors

plt.hist2d(df['D_Pb_hadron']['z'],df['D_Pb_hadron']['theta_lab'],range=([0,1],[0,140]), norm=mcolors.PowerNorm(0.3), bins=100)
plt.show()

In [ ]:
plt.hist2d(df['D_Pb_hadron'].query('pid==-211')['z'],df['D_Pb_hadron'].query('pid==-211')['theta_lab'],range=([0,1],[0,140]), norm=mcolors.PowerNorm(0.3), bins=100)
plt.show()

In [ ]:
plt.hist2d(df['D_Pb_hadron'].query('pid==211')['z'],df['D_Pb_hadron'].query('pid==211')['theta_lab'],range=([0,1],[0,140]), norm=mcolors.PowerNorm(0.3), bins=100)
plt.show()

In [ ]:
plt.hist(df['D_Pb_hadron'].query('pid==-211 and theta_lab>35')['z'],range=(0.0,1.0),bins=100)
plt.yscale('log')